# 变量

声明变量的语法：
```
variable_name data_type [= expression];
```

In [1]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

importing Jupyter notebook from E:\sourcecode\keep_learning\db\pg\pg_00_common.ipynb


In [2]:
# 声明变量
sql = """
do $$ 
declare
   counter    integer = 1;
   first_name varchar(50) = 'John';
   last_name  varchar(50) = 'Doe';
   payment    numeric(11,2) = 20.5;
begin 
   raise notice '% % % has been paid % USD', 
       counter, 
	   first_name, 
	   last_name, 
	   payment;
end $$;
"""
cursor.execute(sql)

1 John Doe has been paid 20.50 USD


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x25775f457a0>

In [3]:
# 变量赋值
sql = """
do $$
declare
	first_name VARCHAR(50);
begin
	first_name = split_part('John Doe',' ', 1);
	raise notice 'The first name is %', first_name;
end;
$$;
"""
cursor.execute(sql)

The first name is John


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x25775f457a0>

In [4]:
# 变量初始化时机
sql = """
do $$ 
declare
   created_at time = clock_timestamp();
begin 
   raise notice '%', created_at;
   perform pg_sleep(3);
   raise notice '%', created_at;
end $$;
"""
cursor.execute(sql)

07:55:30.949386
07:55:30.949386


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x25775f457a0>

## 拷贝数据类型
```
variable_name table_name.column_name%type;
```

In [5]:
sql = """
do 
$$
declare
   film_title film.title%type;
   featured_title film_title%type;
begin 
   -- get title of the film id 100
   select title
   from film
   into film_title
   where film_id = 100;
   
   -- show the film title
   raise notice 'Film title id 100: %s', film_title;
end;
$$;
"""
cursor.execute(sql)

Film title id 100: Brooklyn Deserts


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x25775f457a0>

In [6]:
# 块或子块中的变量
sql = """
do 
$$ 
<<outer_block>>
declare
  counter integer := 0;
begin
   counter := counter + 1;
   raise notice 'The current value of the counter is %', counter;

   declare
       counter integer := 0;
   begin
       counter := counter + 10;
       raise notice 'Counter in the subblock is %', counter;
       raise notice 'Counter in the outer block is %', outer_block.counter;
   end;

   raise notice 'Counter in the outer block is %', counter;
   
end outer_block $$;
"""
cursor.execute(sql)

The current value of the counter is 1
Counter in the subblock is 10
Counter in the outer block is 1
Counter in the outer block is 1


<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x25775f457a0>